%%% Imports

In [1]:

from __future__ import annotations
from time import time
from BrisbaneVPRDatasetSpeed import BrisbaneVPRDatasetSpeed
from VPRNetwork import VPRNetwork
from plotting import plot_confusion_matrix, plot_match_images, plot_gps
from constants import brisbane_event_traverses, brisbane_event_traverses_aliases
from utils import get_short_traverse_name

import os, sys
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib import animation
from scipy import signal
from datetime import datetime

import numpy as np

import lava.lib.dl.slayer as slayer
from lava.lib.dl.slayer.classifier import Rate


In [2]:
# Training settings
from cgi import test


epochs = 50
batch_size = 10

# Network settings
input_size = 34
threshold = 1.0

# Data settings
train_traverse = brisbane_event_traverses[0]
test_traverse = brisbane_event_traverses[2]
train_name = brisbane_event_traverses_aliases[get_short_traverse_name(train_traverse)]
test_name = brisbane_event_traverses_aliases[get_short_traverse_name(test_traverse)]
num_places = 16
start_dist = 3200
place_gap = 50 #164/num_places # The streams run for approximately 164 seconds 
samples_per_sec = 1000
place_length = 27
max_spikes = None

# Sequencer settings
sequence_length = 3

# Plot settings
redo_plot = False
vmin = 0
vmax = 0.05

def transpose( matrix):
    if len(matrix) == 0:
        return []
    return [[matrix[i][j] for i in range(len(matrix))] for j in range(len(matrix[0]))]


# Make a folder for the trained network
trained_folder = 'Trained'
os.makedirs(trained_folder, exist_ok=True)

# Use GPU
print(torch.cuda.is_available())
device = torch.device('cuda')

#---------------- Create the Network -----------------#

# Create the network
net = VPRNetwork(input_size, num_places, threshold=threshold).to(device)


True


In [3]:
# Load the data
training_set = BrisbaneVPRDatasetSpeed(train_traverse, train=True, place_length = place_length, place_gap=place_gap, num_places=num_places, start_dist=start_dist,samples_per_sec=samples_per_sec, max_spikes=max_spikes)
testing_set  = BrisbaneVPRDatasetSpeed(test_traverse, train=False, place_length = place_length, training_locations=training_set.place_locations, place_gap=place_gap, num_places=num_places, start_dist=start_dist, samples_per_sec=samples_per_sec, max_spikes=max_spikes)
            
train_loader = DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=testing_set , batch_size=batch_size, shuffle=True)

Loading training event streams ...
Adding GPS
Duration: 527.16s (which is 3432338 events)
Distance: 6308.49m (which is 3432338 events)


/home/n10223622/snn-event-vpr/src/BrisbaneVPRDatasetSpeed.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chopped_stream['distance'] -= start_distance


Place: 0
Distance: 26.99m (which is 14277 events)
Duration: 1.68s (which is 14277 events)
Place: 1
Distance: 27.00m (which is 13678 events)
Duration: 1.71s (which is 13678 events)
Place: 2
Distance: 27.00m (which is 25507 events)
Duration: 1.72s (which is 25507 events)
Place: 3
Distance: 27.00m (which is 8155 events)
Duration: 1.72s (which is 8155 events)
Place: 4
Distance: 26.99m (which is 15577 events)
Duration: 1.79s (which is 15577 events)
Place: 5
Distance: 26.98m (which is 11931 events)
Duration: 2.18s (which is 11931 events)
Place: 6
Distance: 27.00m (which is 13208 events)
Duration: 2.31s (which is 13208 events)
Place: 7
Distance: 26.99m (which is 11548 events)
Duration: 2.69s (which is 11548 events)
Place: 8
Distance: 27.00m (which is 11337 events)
Duration: 8.42s (which is 11337 events)
Place: 9
Distance: 27.00m (which is 13284 events)
Duration: 3.07s (which is 13284 events)
Place: 10
Distance: 27.00m (which is 9862 events)
Duration: 2.78s (which is 9862 events)
Place: 11
Dis

/home/n10223622/snn-event-vpr/src/BrisbaneVPRDatasetSpeed.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chopped_stream['distance'] -= dist_at_start_time


Place: 3
Distance: 27.00m (which is 15247 events)
Duration: 4.48s (which is 15247 events)
Place: 4
Distance: 26.99m (which is 36449 events)
Duration: 5.92s (which is 36449 events)
Place: 5
Distance: 27.00m (which is 9358 events)
Duration: 3.52s (which is 9358 events)
Place: 6
Distance: 27.00m (which is 24244 events)
Duration: 3.47s (which is 24244 events)
Place: 7
Distance: 26.99m (which is 16644 events)
Duration: 3.42s (which is 16644 events)
Place: 8
Distance: 27.00m (which is 21000 events)
Duration: 3.13s (which is 21000 events)
Place: 9
Distance: 27.00m (which is 26122 events)
Duration: 3.26s (which is 26122 events)
Place: 10
Distance: 26.99m (which is 11166 events)
Duration: 5.79s (which is 11166 events)
Place: 11
Distance: 27.00m (which is 20800 events)
Duration: 21.04s (which is 20800 events)
Place: 12
Distance: 27.00m (which is 13328 events)
Duration: 2.81s (which is 13328 events)
Place: 13
Distance: 27.00m (which is 4796 events)
Duration: 2.38s (which is 4796 events)
Place: 14

In [4]:
# Define an optimiser 
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Training the network
error = slayer.loss.SpikeRate(true_rate=0.2, false_rate=0.03, reduction='sum').to(device)

# Create a training assistant object
stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(net, error, optimizer, stats, classifier=slayer.classifier.Rate.predict)


if redo_plot:
    for i, (input, label) in enumerate(train_loader): # training loop
        output = assistant.train(input, label)

else: 
    for epoch in range(epochs):

        for i, (input, label) in enumerate(train_loader): # training loop
            output = assistant.train(input, label)
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        for i, (input, label) in enumerate(test_loader): # training loop
            output = assistant.test(input, label)
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        if epoch%20 == 19: # cleanup display
            print('\r', ' '*len(f'\r[Epoch {epoch:2d}/{epochs}] {stats}'))
            stats_str = str(stats).replace("| ", "\n")
            print(f'[Epoch {epoch:2d}/{epochs}]\n{stats_str}')

        if stats.testing.best_accuracy:
            torch.save(net.state_dict(), trained_folder + '/network.pt')
        stats.update()
        stats.save(trained_folder + '/')
        net.grad_flow(trained_folder + '/')

[Epoch  0/50] Train loss =     0.85942 | Test  loss =     0.42351                        accuracy = 0.18750
Accuracy was none

Accuracy was none
[Epoch  1/50] Train loss =     0.30336 (min =     0.85942)    accuracy = 0.31250  | Test  loss =     0.42065 (min =     0.42351)    accuracy = 0.06250 (max = 0.18750)
Accuracy was none
[Epoch  2/50] Train loss =     0.29936 (min =     0.30336)    accuracy = 0.25000 (max = 0.31250) | Test  loss =     0.43928 (min =     0.42065)    accuracy = 0.06250 (max = 0.18750)
Accuracy was none
[Epoch  3/50] Train loss =     0.27028 (min =     0.29936)    accuracy = 0.43750 (max = 0.31250) | Test  loss =     0.42071 (min =     0.42065)    accuracy = 0.25000 (max = 0.18750)
Accuracy was none
[Epoch  4/50] Train loss =     0.24419 (min =     0.27028)    accuracy = 0.68750 (max = 0.43750) | Test  loss =     0.49894 (min =     0.42065)    accuracy = 0.12500 (max = 0.25000)
Accuracy was none
[Epoch  5/50] Train loss =     0.24357 (min =     0.24419)    accuracy

In [5]:
# import the best network during training 
net.load_state_dict(torch.load(trained_folder + '/network.pt'))
net.export_hdf5(trained_folder + '/network.net')

# Get the output for the input to each place
test_loader2  = DataLoader(dataset=testing_set , batch_size=batch_size, shuffle=False)
rate = []
labels = []
for i, (input, label) in enumerate(test_loader2):
    output = net(input.to(device)) # Get network output
    #guesses = assistant.classifier(output).cpu().data.numpy() # get the predictions 
    rate.extend(Rate.rate(output).cpu().data.numpy()) # Get the firing rates for each place 
    labels.extend(label.cpu().data.numpy()) # Get place labels


# # Get rates in percentages of total
# for i in range(num_places):
#     sum = np.sum(rate[i])
#     if sum != 0:
#         rate[i] = np.divide(rate[i],sum)

# Make confusion matrix annotations
accuracy = 0
matches = []
annotations = [['' for i in range(num_places)] for j in range(num_places)]
for qryIndex in range(num_places):
    max_idx = np.argmax(rate[qryIndex])
    matches.append(max_idx)
    annotations[max_idx][qryIndex] = 'x'
    if max_idx ==qryIndex:
        accuracy += 1


#--------------- Apply a sequencer ------------------#
I = np.identity(sequence_length)
conv = signal.convolve2d(rate, I, mode='same')
print(np.shape(conv))

# Make confusion matrix annotations
accuracy_s = 0
matches_with_seq = []
annotations_s = [['' for i in range(num_places)] for j in range(num_places)]
for qryIndex in range(num_places):
    max_idx = np.argmax(conv[qryIndex])
    matches_with_seq.append(max_idx)
    annotations_s[max_idx][qryIndex] = 'x'
    if max_idx == qryIndex:
        accuracy_s += 1



(16, 16)


In [6]:

#--------------- Save Results ------------------#

# Make new folder for results
from tracemalloc import start
from plotting import plot_gps


time_stamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
results_path = "./../results/" + time_stamp
os.mkdir(results_path)


# Save query and match images 
images_path = results_path + "/matched_images"
images_path_seq = results_path + "/matched_images_seq"
os.mkdir(images_path)
os.mkdir(images_path_seq)

print('help')
plot_match_images(images_path + "/", matches, training_set.place_images, testing_set.place_images)
plot_match_images(images_path_seq + "/", matches_with_seq, training_set.place_images, testing_set.place_images)

# Save the confusion matrices
confusion_path = results_path + "/confusion_matrices"
os.mkdir(confusion_path)
rate = transpose(rate)
conv = transpose(conv)
output_path = confusion_path + "/confusion_matrix" 
output_path_s = confusion_path + "/confusion_matrix_seq" 
plot_confusion_matrix(rate, labels, annotations, output_path, vmin, vmax)
plot_confusion_matrix(conv, labels, annotations_s, output_path_s, vmin, vmax)

# Save GPS map'
gps_path = results_path + "/gps_locations"
plot_gps(gps_path, training_set.place_locations, testing_set.place_locations)

# Save test settings and accuracy
accuracy = accuracy/num_places
accuracy_s = accuracy_s/num_places


help


<Figure size 1296x288 with 0 Axes>

<Figure size 1296x288 with 0 Axes>

In [7]:

log_path = results_path + "/log.txt"
log_string  = """
---- DATA SETTINGS ---- 
Training datasets = {}
Testing datasets = {} 
# Places = {}
Start dist = {} [m]
Place gap = {} [m]
Place length = {} [m]
Samples per second = {} 
Max spikes per sample = {}
---- NETWORK SETTINGS ---- 
Input size = {}x{} 
Threshold = {}
---- TRAINING SETTINGS ---- 
Epochs = {}
Batch size = {}
---- SEQUENCER SETTINGS ----
Sequence lengtth = {} 
---- RESULTS ----
Accuracy = {}
Accuracy (sequencer) = {}
""".format(train_name,test_name,num_places,start_dist,place_gap,place_length,
                samples_per_sec,max_spikes,input_size,input_size,threshold,epochs,
                batch_size,sequence_length,accuracy,accuracy_s)
f = open(log_path,'w')
f.write(log_string)
f.close()

print("The accuracy of the network is: " + str(accuracy))
print("The accuracy with a sequencer is: " + str(accuracy_s))

The accuracy of the network is: 0.1875
The accuracy with a sequencer is: 0.4375
